`py_grama`

---

A demonstration

In [ ]:
# Setup
import grama as gr
from grama.models import make_cantilever_beam

md_beam = make_cantilever_beam()

# Ontology

In [ ]:
# Models have both *functions* and *distributions*
md_beam.printpretty()

# Model Analysis


In [ ]:
# The model has a function + distribution;
# Monte Carlo is now a one-liner
df_mc = md_beam >> gr.ev_monte_carlo(n=100, df_det="nom")

df_mc.describe()

In [ ]:
# The resulting data have model context for plotting
df_mc >> gr.pt_auto()

In [ ]:
# Utilities for sensitivity analysis
md_beam >> gr.ev_hybrid(n=500, df_det="nom") >> gr.tf_sobol()

In [ ]:
# Graphical exploration
md_beam >> gr.ev_sinews(df_det="nom", skip=True) >> gr.pt_auto()
# rm `skip` for evaluation

# Model Building

In [ ]:
# Models are built in stages
md_test = (
    gr.Model("My model")
    >> gr.cp_function(fun=lambda x: x[0] + x[1], var=2, out=1)
    >> gr.cp_bounds(x0=(0, 1))
    >> gr.cp_marginals(x1={"dist": "uniform", "loc": 0, "scale": 1})
)
md_test.printpretty()

In [ ]:
# Cannot perform MC without an explicit dependence structure!
# md_test >> gr.ev_monte_carlo(df_det="nom")

In [ ]:
# We can update a model; this allows us to
# build up in stages
md_test = md_test >> gr.cp_copula_independence()
md_test.printpretty()
md_test >> gr.ev_monte_carlo(df_det="nom", seed=101)

In [ ]:
# We can fit distributions from data...
from grama.data import df_stang

md_stang = (
    gr.Model("E, mu distribution")
    >> gr.cp_marginals(
        E=gr.marg_named(df_stang.E, "norm"),
        mu=gr.marg_named(df_stang.mu, "beta"),
    )
    >> gr.cp_copula_gaussian(df_data=df_stang)
)
md_stang.printpretty()

In [ ]:
# and we can simulate from those fits
df_stang_synthetic = md_stang >> gr.ev_monte_carlo(
    n=1e3, df_det="nom", skip=True
)
df_stang_synthetic >> gr.pt_auto()

In [ ]:
# Some UQ knowledge is built into the tools
# md_stang >> gr.ev_hybrid(n=500, df_det="nom")

# Surrogate / Metamodels

In [ ]:
# Can build metamodels on data;
# fit a gaussian process
from grama.fit import fit_gp

md_beam_meta = fit_gp(df_mc, md=md_beam)
# Metamodel inherits model distribution
md_beam_meta.printpretty()

In [ ]:
# Perform qualitative checks on a learned model
md_beam_meta >> gr.ev_sinews(df_det="nom") >> gr.pt_auto()

In [ ]:
# fit_gp is a wrapper; we can use a scikit-learn kernel
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as Con

kernel = Con(1, (1e-3, 1e3)) * RBF([1] * 6, (1e-8, 1e8))

# Layered options with sensible defaults
md_beam_custom = fit_gp(df_mc, md=md_beam, kernel=kernel)
md_beam_custom.printpretty()